In [73]:
import cv2
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
from scipy.spatial import distance
import warnings
warnings.simplefilter(action='ignore')

In [74]:
def read_image_colors(image_path):
    altura, largura = 250, 250
    imagem = cv2.imread(image_path)
    imagem = cv2.resize(imagem, (largura, altura))
    imagem_rgb = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)

    cores_encontradas = []
    for linha in imagem_rgb:
        for pixel in linha:
            cor = tuple(pixel)
            cores_encontradas.append(cor)

    return cores_encontradas


imagem_path = r'D:\testekeras\cozinha.jpg'


cores_da_imagem = read_image_colors(imagem_path)


dados = pd.read_csv(r"D:\testekeras\all_cores.csv", sep=";", encoding="latin-1")


def converter_codigo_rgb(valor):
    try:
        valor = valor.replace('RGB', '').strip()
        valores = tuple(map(int, valor.split(',')))
        return valores
    except ValueError:
        return None

dados['CodigoRgb'] = dados['CodigoRgb'].apply(converter_codigo_rgb)


dados = dados.dropna(subset=['CodigoRgb'])


num_clusters = 5


cores_imagem_array = np.array(cores_da_imagem)


kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(cores_imagem_array)


centroides = kmeans.cluster_centers_


def encontrar_cor_mais_proxima(cor, cores_df):
    distancias = [distance.euclidean(cor, c) for c in cores_df]
    indice_cor_mais_proxima = np.argmin(distancias)
    return cores_df[indice_cor_mais_proxima]


cores_proximas = [encontrar_cor_mais_proxima(centro, dados['CodigoRgb'].to_list()) for centro in centroides]


for i, cor_proxima in enumerate(cores_proximas):
    cluster = i + 1
    cor_dataframe = tuple(cor_proxima)
    nome_cor_dataframe = dados[dados['CodigoRgb'].apply(lambda x: tuple(x)) == cor_dataframe]['CorTraduzida'].values[0]
    print(f'Cluster {cluster}: Cor do Cluster: {cor_proxima}, Cor do DataFrame: {cor_dataframe}, Nome da Cor: {nome_cor_dataframe}')


Cluster 1: Cor do Cluster: (44, 46, 50), Cor do DataFrame: (44, 46, 50), Nome da Cor: RUINED SMORES
Cluster 2: Cor do Cluster: (132, 132, 130), Cor do DataFrame: (132, 132, 130), Nome da Cor: Chicana
Cluster 3: Cor do Cluster: (218, 218, 218), Cor do DataFrame: (218, 218, 218), Nome da Cor: Cinzento carvão
Cluster 4: Cor do Cluster: (168, 170, 171), Cor do DataFrame: (168, 170, 171), Nome da Cor: Basalto
Cluster 5: Cor do Cluster: (103, 102, 98), Cor do DataFrame: (103, 102, 98), Nome da Cor: Pomba Cinza


In [75]:
dados

,Cor,Codigoa,CodigoRgb,CorTraduzida
0,Black,0,"(0, 0, 0)",Preto
1,grey11,1C1C1C,"(28, 28, 28)",cinza11
2,grey21,363636,"(54, 54, 54)",cinza21
3,grey31,4F4F4F,"(79, 79, 79)",cinza31
4,DimGray,696969,"(105, 105, 105)",Cinza Escuro
...,...,...,...,...
1172,Daisy,E2E1DB,"(239, 237, 227)",Margarida
1173,Bluish White,F4F0EC,"(246, 244, 241)",Branco azulado
1174,Milk,F2EBDD,"(227, 227, 220)",Leite
1175,Natural White,F2F0E8,"(236, 229, 218)",Branco natural


In [51]:

df_treino = pd.read_csv(r'D:\testekeras\treino.csv',sep=";",encoding="latin-1")  


df_avaliacao = pd.read_csv(r'D:\testekeras\avaliacao.csv',sep=";",encoding="utf-8")
df_avaliacao

,nome,imagem
0,Cozinha Completa Madesa Reims 310001 com Armár...,https://static.mobly.com.br/r/2000x2000/p/Made...
1,Conjunto de Mesa com 6 Cadeiras Granada Branco...,https://static.mobly.com.br/r/2000x2000/p/Fabo...
2,Mesa de Jantar Retangular com Tampo de Vidro V...,https://static.mobly.com.br/r/2000x2000/p/Art-...
3,Mesa de Jantar Redonda Spacy Off White e Café ...,https://static.mobly.com.br/r/2000x2000/p/Brig...
4,Mesa de Jantar Extensível Retangular Marea Pre...,https://static.mobly.com.br/r/2000x2000/p/Mobl...


In [76]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from PIL import Image
import numpy as np
from sklearn.cluster import KMeans
import requests
from io import BytesIO


def extract_colors_from_image(image_url, num_colors=5):
   
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content))
    
   
    image = image.resize((100, 100))
    
    
    image_array = np.array(image)
    

    pixels = image_array.reshape(-1, 3)
    
   
    kmeans = KMeans(n_clusters=num_colors, random_state=0).fit(pixels)
    
  
    cluster_centers = kmeans.cluster_centers_
    

    cluster_centers = cluster_centers.astype(int)

    colors_hex = ['#%02x%02x%02x' % (r, g, b) for r, g, b in cluster_centers]
    
    return colors_hex


def extract_features_from_image(image_url):
   
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content))
    

    width, height = image.size
    
  
    aspect_ratio = width / height
    
  
    features = {
        'largura': width,
        'altura': height,
        'proporcao_aspecto': aspect_ratio
       
    }
    
    return features


df_treino = pd.read_csv(r'D:\testekeras\treino.csv', sep=";", encoding="utf-8") 


df_avaliacao = pd.read_csv(r'D:\testekeras\avaliacao.csv', sep=";", encoding="utf-8")


df_treino['cores_imagem'] = df_treino['URL_imagem'].apply(extract_colors_from_image)
df_treino['caracteristicas_imagem'] = df_treino['URL_imagem'].apply(extract_features_from_image)


label_encoder = LabelEncoder()
df_treino['Ambiente_encoded'] = label_encoder.fit_transform(df_treino['Ambiente'])
df_treino['Produto_encoded'] = label_encoder.fit_transform(df_treino['Tipo'])


onehot_encoder = OneHotEncoder(sparse=False)
y_ambiente_treino_onehot = onehot_encoder.fit_transform(df_treino['Ambiente_encoded'].values.reshape(-1, 1))
y_produto_treino_onehot = onehot_encoder.fit_transform(df_treino['Produto_encoded'].values.reshape(-1, 1))


X_treino = df_treino[['nome', 'descricao', 'cores_imagem', 'caracteristicas_imagem']]
X_train, X_val, y_ambiente_treino, y_ambiente_val, y_produto_treino, y_produto_val = train_test_split(
    X_treino, y_ambiente_treino_onehot, y_produto_treino_onehot, test_size=0.2, random_state=42)

max_words = 1000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train['descricao'])

X_treino_seq = tokenizer.texts_to_sequences(X_train['descricao'])
X_val_seq = tokenizer.texts_to_sequences(X_val['descricao'])

input_length = 505 

X_treino_pad = pad_sequences(X_treino_seq, maxlen=input_length)
X_val_pad = pad_sequences(X_val_seq, maxlen=input_length)


model_ambiente = Sequential()
model_ambiente.add(Embedding(input_dim=max_words, output_dim=128, input_length=input_length))
model_ambiente.add(LSTM(64, return_sequences=True))
model_ambiente.add(GlobalMaxPooling1D())
model_ambiente.add(Dense(64, activation='relu'))
model_ambiente.add(Dense(y_ambiente_treino.shape[1], activation='softmax')) 

model_ambiente.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


model_ambiente.fit(X_treino_pad, y_ambiente_treino, epochs=5, batch_size=32, validation_data=(X_val_pad, y_ambiente_val))


ambiente_predictions = model_ambiente.predict(X_val_pad)


model_produto = Sequential()
model_produto.add(Embedding(input_dim=max_words, output_dim=128, input_length=input_length))
model_produto.add(LSTM(64, return_sequences=True))
model_produto.add(GlobalMaxPooling1D())
model_produto.add(Dense(64, activation='relu'))
model_produto.add(Dense(y_produto_treino.shape[1], activation='softmax'))  # Número de classes

model_produto.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


model_produto.fit(X_treino_pad, y_produto_treino, epochs=5, batch_size=32, validation_data=(X_val_pad, y_produto_val))


produto_predictions = model_produto.predict(X_val_pad)


for imagem_url in df_treino['URL_imagem']:
    cores_da_imagem = extract_colors_from_image(imagem_url)
    print("Cores da Imagem:", cores_da_imagem)


for imagem_url in df_treino['URL_imagem']:
    caracteristicas_da_imagem = extract_features_from_image(imagem_url)
    print("Características da Imagem:", caracteristicas_da_imagem)



Epoch 1/5
1/1 [==============================] - 2s 2s/step - loss: 1.0997 - accuracy: 0.0000e+00 - val_loss: 1.1044 - val_accuracy: 0.0000e+00
Epoch 2/5
1/1 [==============================] - 0s 185ms/step - loss: 1.0761 - accuracy: 0.7500 - val_loss: 1.1237 - val_accuracy: 0.0000e+00
Epoch 3/5
1/1 [==============================] - 0s 192ms/step - loss: 1.0551 - accuracy: 0.7500 - val_loss: 1.1417 - val_accuracy: 0.0000e+00
Epoch 4/5
1/1 [==============================] - 0s 188ms/step - loss: 1.0366 - accuracy: 0.7500 - val_loss: 1.1622 - val_accuracy: 0.0000e+00
Epoch 5/5
1/1 [==============================] - 0s 344ms/step
Epoch 1/5
1/1 [==============================] - 2s 2s/step - loss: 1.1024 - accuracy: 0.2500 - val_loss: 1.0853 - val_accuracy: 1.0000
Epoch 2/5
1/1 [==============================] - 0s 173ms/step - loss: 1.0828 - accuracy: 1.0000 - val_loss: 1.0654 - val_accuracy: 1.0000
Epoch 3/5
1/1 [==============================] - 0s 165ms/step - loss: 1.0687 - accuracy: